In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor

import time
from numba import jit
import itertools
import random   
import time
import warnings
import gc
import math

warnings.filterwarnings("ignore")
from sklearn.linear_model import lasso_path

In [ ]:
# Experiment Functions
def load_openml(data_id,ordinal = True, y_label = ''):
    """Load dataset by id from OpenML. If ordinal == True, encode categorical columns 
    via ordinal encoding. If ordinal == False then encode categorical columns with dummy vars.
    """
    
    dataset1 = sklearn.datasets.fetch_openml(data_id = data_id,as_frame = True)
    name = dataset1.details['name']
    X, y = dataset1.data, dataset1.target 
    data = pd.DataFrame(X,columns = dataset1.feature_names)

    if len(y_label) == 0:
        data['y'] = y
    else:
        data['y'] = y[y_label]

    #shuffle index
    data = data.sample(frac = 1)
    y = data['y']
    y = y.astype(float)
    X = data.drop('y',axis = 1)

    #encode categorical columns
    cat = list(set(X.columns) - set(X.select_dtypes(include=np.number).columns.tolist()))
    
    if ordinal == True:
        for col in cat:
            X[col] = X[col].astype('category').cat.codes
            X[col] = X[col].fillna(max(X[col]+1))
        
    elif ordinal == False:
        X = pd.get_dummies(X,columns = cat)
    
        
    #impute median
#    X = X.fillna(X.median()) 
    return X,y,name

def get_node_depths(tree1):
    """
    Get the node depths of the decision tree

    >>> d = DecisionTreeClassifier()
    >>> d.fit([[1,2,3],[4,5,6],[7,8,9]], [1,2,3])
    >>> get_node_depths(d.tree_)
    array([0, 1, 1, 2, 2])
    """
    def get_node_depths_(current_node, current_depth, l, r, depths):
        depths += [current_depth]
        if l[current_node] != -1 and r[current_node] != -1:
            get_node_depths_(l[current_node], current_depth + 1, l, r, depths)
            get_node_depths_(r[current_node], current_depth + 1, l, r, depths)

    depths = []
    get_node_depths_(0, 0, tree1.tree_.children_left, tree1.tree_.children_right, depths) 
    return np.array(depths)

def get_node_count(tree_list,best_vars):
    num_nodes = 0
    depths = np.sum(best_vars,axis = 1)
    for i in range(len(best_vars)):
        tree1 = tree_list[i]
        node_depths = get_node_depths(tree1)
        depth_cutoff = depths[i]
        if depth_cutoff > 0:
            num_nodes = num_nodes + sum(node_depths <= depth_cutoff)
    return num_nodes


In [ ]:
def difference_array_list(X,tree_list):
    diff_array_list = []
    for tree1 in tree_list:
        diff_array_list.append(difference_array(X,tree1))
    return np.array(diff_array_list)

def difference_array(X, tree_learner):
    """function that takes a decision tree and returns an 
    [m,d]
    Each row is an instance and each column is a depth level.
    We take the difference in internal node values to get the delta for each depth level.
    the column sum of the output is the prediction of the tree
    """
    
    node_indicator = tree_learner.decision_path(X)
    values = tree_learner.tree_.value
    vdiffs = []
    
    for i in range(0,len(X)):
        node_ids = node_indicator.indices[node_indicator.indptr[i] : node_indicator.indptr[i + 1]]
        instance_values = np.ndarray.flatten(values[node_ids])
        diffs = [j-i for i, j in zip(instance_values[:-1], instance_values[1:])]
        row = np.zeros(tree_learner.max_depth)
        row[:len(diffs)] = diffs
        vdiffs.append(row)
        
    return np.array(vdiffs)

@jit(nopython=True)
def evaluate_test_error(difference_array_list,Y,vars_z,learning_rate):
    pred = np.zeros(len(Y))
    for i in range(len(vars_z)):
        pred += np.dot(difference_array_list[i],vars_z[i])*learning_rate     
    return np.square(np.subtract(Y, pred)).mean()
  



In [ ]:
from numba import jit
import itertools
import random    
@jit(nopython=True)
def precompute_predictions(diff_array_list,temp_vars,learning_rate,cycle_ind):
    
    precompute_pred = np.zeros(len(diff_array_list[0]))    
    for i in range(len(diff_array_list)):
        if i != cycle_ind:
            precompute_pred += np.dot(diff_array_list[i],temp_vars[i])*learning_rate 
   
    return precompute_pred

@jit(nopython=True)
def evaluate_candidates(diff_array_list,temp_vars,learning_rate,cycle_ind,candidates,
                        precompute_pred,Y,alpha,W_array, normalization):
    scores = []
    for candidate in candidates:
        temp_vars[cycle_ind] = candidate
        pred_candidate = np.dot(diff_array_list[cycle_ind],candidate)*learning_rate
        pred = np.add(precompute_pred,pred_candidate)
        err = np.sum((Y-pred)**2)/len(Y) + (alpha/normalization)*np.sum(np.dot(W_array[cycle_ind],candidate))
        scores.append(err)
    return scores

@jit(nopython=True)
def eval_obj(Y,diff_array_list,vars_z,learning_rate,alpha,W_array,normalization):
    pred = np.zeros(len(Y))
    regularization = 0
    for i in range(len(vars_z)):
        pred+= learning_rate*np.dot(diff_array_list[i],vars_z[i])
        regularization += np.sum(np.dot(W_array[i],vars_z[i]))
    
    bias = np.sum((Y-pred)**2)/len(Y)
    
    return bias + regularization*alpha/normalization

@jit(nopython=True)
def converge_test(sequence, threshold,tail_length):
    diff = np.diff(sequence)
    if len(diff) < (tail_length+1):
        return False
    else:
        return (np.max(np.abs(diff[-tail_length:])) < threshold)


def solve_weighted(Y,tree_list,diff_array_list,alpha,learning_rate,
                                          W_array,normalization,warm_start= []):
    max_depth = tree_list[0].max_depth
    Y = np.array(Y.values)
    
    vars_z = np.zeros((len(tree_list),max_depth))
    if len(warm_start) > 0:
        vars_z = np.array(warm_start)
    
    candidates = np.vstack([np.zeros(max_depth),np.tril(np.ones((max_depth,max_depth)))])
    
    convergence_scores = np.array([])
    converged = False
    ind_counter = 0
    local_best = 9999
    total_inds = 0
    while converged == False:
        
        cycle_ind = ind_counter % len(vars_z)   

        temp_vars= vars_z.copy()
        precompute_pred = precompute_predictions(diff_array_list,temp_vars,learning_rate,cycle_ind)
        scores = evaluate_candidates(diff_array_list,temp_vars,learning_rate,cycle_ind,
                                     candidates,precompute_pred,Y,alpha,W_array,normalization)
        
        vars_z[cycle_ind] = candidates[np.argmin(scores)]
        convergence_scores = np.append(convergence_scores,eval_obj(Y,diff_array_list,
                                                                   vars_z,learning_rate,alpha,W_array,normalization))
        converged = converge_test(np.array(convergence_scores),10**-6,3)
        
        ind_counter = ind_counter + 1
        total_inds = total_inds + 1
        
        #local search
        if converged == True:
            support_indicies = np.where(~np.all(vars_z == 0, axis=1))[0]
            zero_indicies = np.where(np.all(vars_z == 0, axis=1))[0]
            
            if convergence_scores[-1] > local_best:
                converged = True
            
            elif len(support_indicies)> 0:
                local_ind = random.choice(support_indicies)
                vars_z[local_ind] = np.zeros(max_depth)
                
                if len(zero_indicies) > 0:
                    ind_counter = min(zero_indicies)
                    converged = False
                    local_best = convergence_scores[-1]
                
                else:
                    converged = True
     
    return vars_z , total_inds


In [ ]:
# Weight Penalties

def nodes_per_layer(tree_list):
    max_depth = tree_list[0].max_depth
    results = []
    for tree1 in tree_list:
        depths = get_node_depths(tree1)
        values,counts = np.unique(depths,return_counts = True)
        diag = np.zeros(max_depth)
        counts = counts[1:]
        diag[:len(counts)] = counts
        results.append(np.diag(diag))
    
    return np.array(results)

def total_nodes(tree_list):
    return np.sum(tree1.tree_.node_count for tree1 in tree_list) - len(tree_list)

def prune_polish(difference_array_list,Y,vars_z,learning_rate):
    pred_array = []
    for i in range(len(vars_z)):
        if sum(vars_z[i])>0:
            pred_array.append(np.dot(difference_array_list[i],vars_z[i])*learning_rate)
    
    if len(pred_array) == 0:
        return np.zeros(len(vars_z))
    
    pred_array = np.transpose(pred_array)
    lm = Ridge(alpha = 0.01, fit_intercept = False).fit(pred_array,Y)
    coef = lm.coef_
    return coef

@jit(nopython=True)
def evaluate_test_error_polished(difference_array_list,Y,vars_z,coef,learning_rate):
    pred = np.zeros(len(Y))
    j = 0
    for i in range(len(vars_z)):
        if sum(vars_z[i])>0:
            pred += np.dot(difference_array_list[i],vars_z[i])*learning_rate*coef[j]  
            j+=1
    return np.square(np.subtract(Y, pred)).mean()

# Experiment

In [ ]:
import time
def subensemble_predict(X,tree_list,learning_rate,ntrees):
    pred = np.zeros(len(X))
    for tree1 in tree_list[:ntrees]:
        pred += tree1.predict(X)*learning_rate
    return pred

def get_node_count_all(tree_list):
    num_nodes = 0
    for tree1 in tree_list:
        num_nodes = num_nodes + tree1.tree_.node_count
    return num_nodes

def lasso_predict(X,tree_list,coef):
    pred = np.zeros(len(X))
    for i in range(len(tree_list)):
        pred += tree_list[i].predict(X)*coef[i]
    return pred

import gurobipy as gp
from gurobipy import GRB
from itertools import product

def l0_ensemble_select(features, response,node_count, node_limit, warm_up=None, verbose=False, time_limit = 600):
    """
    Deploy and optimize the MIQP formulation of L0-Regression.
    """
    t1 = time.time()
    assert isinstance(node_limit, (int, np.integer))
    regressor = gp.Model()
    samples, dim = features.shape
    assert samples == response.shape[0]


    # Append a column of ones to the feature matrix to account for the y-intercept
    X = np.concatenate([features, np.ones((samples, 1))], axis=1)  
    
    # Decision variables
    beta = regressor.addVars(dim, lb=-GRB.INFINITY, name="beta") # Weights
 
    # iszero[i] = 1 if beta[i] = 0  
    iszero = regressor.addVars(dim, vtype=GRB.BINARY, name="iszero") 
    
    # Objective Function (OF): minimize 1/2 * RSS using the fact that
    # if x* is a minimizer of f(x), it is also a minimizer of k*f(x) iff k > 0
    Quad = np.dot(X.T, X)
    lin = np.dot(response.T, X)
    obj = sum(0.5 * Quad[i,j] * beta[i] * beta[j]
              for i, j in product(range(dim), repeat=2))
    obj -= sum(lin[i] * beta[i] for i in range(dim))
    obj += 0.5 * np.dot(response, response)
    regressor.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint sets
    for i in range(dim):
        # If iszero[i]=1, then beta[i] = 0
        regressor.addSOS(GRB.SOS_TYPE1, [beta[i], iszero[i]])
        
        
    regressor.addConstr(sum([node_count[i]*(1-iszero[i]) \
                             for i in range(len(node_count))]) <= node_limit) # Budget constraint
    
    # We may use the Lasso or prev solution with fewer features as warm start
    if warm_up is not None and len(warm_up) == dim:
        for i in range(dim):
            iszero[i].start = (abs(warm_up[i]) < 1e-6)    
    if not verbose:
        regressor.params.OutputFlag = 0
    regressor.params.timelimit = time_limit
    regressor.params.mipgap = 0.001
  
    regressor.optimize()

    coeff = np.array([beta[i].X for i in range(dim)])
    t2 = time.time()
    return  coeff, (t2 - t1)



In [ ]:
inds = [528,
505,
196,
547,
531,
223,
541,
41021,
512,
507,
183,
42570,
405
,287
,503
,189
,227
,308
,558
,201,
216,
537,
574]

In [ ]:
import pickle

max_depth = 20
n_estimators = 100
ntree_range = list(range(1,n_estimators,10))

reproduce = []
baseline = []
for i in inds:
    X,y,name = load_openml(i,False)
    y = pd.Series(y)
    y.index = X.index



    learning_rate = 1/n_estimators
    results_all = pd.DataFrame()
    n_splits = 5
    kf = KFold(n_splits=n_splits)

    threshold1 = 0.05
    depths_all = []
    
    print(name)
    k = 0
    for train_index, test_index in kf.split(X):
        k = k+1
        xTrain, xTest = X.iloc[train_index], X.iloc[test_index]
        yTrain, yTest = y.iloc[train_index], y.iloc[test_index]

        xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size=0.25) 

        xTrain = preprocessing.StandardScaler().fit_transform(xTrain)
        xTrain = pd.DataFrame(xTrain,columns = X.columns)
        xVal = preprocessing.StandardScaler().fit_transform(xVal)
        xVal = pd.DataFrame(xVal,columns = X.columns)
        xTest = preprocessing.StandardScaler().fit_transform(xTest)
        xTest = pd.DataFrame(xTest,columns = X.columns)
        yTrain = preprocessing.StandardScaler().fit_transform(yTrain.values.reshape(-1, 1))
        yTest = preprocessing.StandardScaler().fit_transform(yTest.values.reshape(-1, 1))
        yVal = preprocessing.StandardScaler().fit_transform(yVal.values.reshape(-1, 1))

        yTrain = pd.Series(yTrain.flatten())
        yTrain.index = xTrain.index
        yTest = pd.Series(yTest.flatten())
        yTest.index = xTest.index
        yVal = pd.Series(yVal.flatten())
        yVal.index = xVal.index


        model = RandomForestRegressor(max_depth = max_depth,n_estimators = n_estimators,
                                     max_features = 'sqrt', n_jobs = 4).fit(xTrain,yTrain)

        tree_list = np.array(model.estimators_)
        W_array = nodes_per_layer(tree_list)
        normalization = total_nodes(tree_list)
        
        base_err = sklearn.metrics.mean_squared_error(yTest,model.predict(xTest))
        val_err = sklearn.metrics.mean_squared_error(yVal,model.predict(xVal))
        
        base_rf_nodes = 0
        for tree1 in tree_list:
            base_rf_nodes = base_rf_nodes + tree1.tree_.node_count


        diff_array_list = difference_array_list(xTrain,tree_list)
        diff_test_array_list = difference_array_list(xTest,tree_list)
        diff_val_array_list = difference_array_list(xVal,tree_list)

        alphas = []
        warm_start = []

        results = []

        for alpha in np.flip(np.logspace(-2,1.5,50)):
            
            vars1,iters = solve_weighted(yTrain,tree_list,diff_array_list,
                                    alpha,learning_rate,W_array,normalization, warm_start = warm_start)
            warm_start = vars1
            coef = prune_polish(diff_array_list,yTrain,vars1,learning_rate)
            err = evaluate_test_error_polished(diff_val_array_list,yVal.values,vars1,
                                                          coef,learning_rate)
            results.append([alpha,err,iters])

        results_df = pd.DataFrame(results,columns = ['alpha', 'err','iter'])


        list_errs = []
        list_nodes = []
        list_threshold = []
        
        for i in [0.01,0.025,0.05]:
            UL = val_err + i
            best_alpha = np.max(results_df[results_df['err']<UL]['alpha'])
            if np.isnan(best_alpha):
                best_alpha = results_df['alpha'].min()
            print(best_alpha)
            
            
            vars_best,_ = solve_weighted(yTrain,tree_list,diff_array_list,best_alpha,learning_rate,
                                        W_array,normalization,)

            coef_best = prune_polish(diff_array_list,yTrain,vars_best,learning_rate)

            pruned_err = evaluate_test_error_polished(diff_test_array_list,yTest.values,vars_best,
                                                              coef_best,learning_rate)
            pruned_rf_nodes = get_node_count(tree_list,vars_best)
            nodes_limit = pruned_rf_nodes
            prune_score = pruned_err
            
            
            # baseline
            base_results = []
            for ntrees in ntree_range:
                pred_test = subensemble_predict(xVal,tree_list,learning_rate,ntrees)
                score = mean_squared_error(yVal,pred_test)
                nnodes = get_node_count_all(tree_list[:ntrees])
                base_results.append([ntrees,score,nnodes])
                
            base_results = pd.DataFrame(base_results,columns = ['ntrees','score','nnodes'])
            best_trees = base_results[base_results['nnodes']<=nodes_limit]['ntrees'].max()
            
            if not math.isnan(best_trees):
                pred_test = subensemble_predict(xTest,tree_list,learning_rate,best_trees)
            
            else:
                pred_test = np.repeat(yTrain.mean(),len(yTest))
            
            base_score = mean_squared_error(yTest,pred_test)
            
            #lasso
            pred_array = []
            for tree1 in tree_list:
                pred_array.append(tree1.predict(xTrain))
            pred_array = np.transpose(pred_array)
            alphas, coef_path, _ = lasso_path(pred_array, yTrain ,n_alphas = 100, eps = 10**-10)
            coef_path = np.transpose(coef_path) 

            lasso_results = []
            for coef in coef_path:
                subforest = np.array(tree_list)[coef > 0]
                lasso_pred = lasso_predict(xVal,subforest,coef[coef>0])
                lasso_score = mean_squared_error(yVal,lasso_pred)
                lasso_nnodes = get_node_count_all(subforest)
                lasso_ntrees = sum(coef>0)
                lasso_results.append([lasso_ntrees,lasso_score,lasso_nnodes])
                
                
            lasso_results = pd.DataFrame(lasso_results,columns = ['ntrees','score','nnodes'])
            
            if sum(lasso_results['nnodes'] <= nodes_limit) != 0:
                best_coef = coef_path[lasso_results[lasso_results['score']\
                  == lasso_results[lasso_results['nnodes']<=nodes_limit]['score'].min()].index[0]]
                best_subforest = np.array(tree_list)[best_coef > 0]
                lasso_pred = lasso_predict(xTest,best_subforest,best_coef[best_coef>0])
                lasso_score = mean_squared_error(yTest, lasso_pred)
            else:
                best_coef = np.zeros(len(tree_list))
                lasso_score = 1.0
            
            ##### L0
            node_count = np.array([tree1.tree_.node_count for tree1 in tree_list])
            pred_test = np.transpose([tree1.predict(xTest) for tree1 in tree_list])
            
            
            l0_coef,l0_time = l0_ensemble_select(pred_array,
                                                 yTrain,node_count,node_limit = nodes_limit,
                                                  warm_up = best_coef ,
                                                 time_limit = 60)
            l0_score = mean_squared_error(yTest,pred_test@l0_coef)
            
            ##### Store Results
            
            baseline.append([name,k,i,prune_score,base_score,lasso_score,l0_score,l0_time])
            reproduce.append([name,k,i,nodes_limit,train_index, test_index ])
            
            print([name,k,i,prune_score,base_score,lasso_score,l0_score,l0_time])
            